In [1]:
#=======================================#
# Yes, this notebook is over-commented. #
#=======================================#

In [2]:
# Make notebook span entire screen, horizontally.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import gym
import numpy as np
import tensorflow as tf

In [4]:
# TODO:
#  -> fix loss funtion
#  -> check reward processing

class PolicyAgent(object):
    def __init__(self):
        self.n_actions = 4
        
        # Build graph (i.e. internal neural network).
        self._build()
        
        # Create environment within which the graph will train.
        self.sess = tf.Session()
        self.saver = tf.train.Saver()
        
        # Evaluate an tf.Operation (returned by global_variables_initializer()) that initializes TRAINABLE VARIABLES.
        self.sess.run(tf.global_variables_initializer())
        
    def _build(self):
        self.obs      = tf.placeholder(tf.float32, (None, 8))
        self.acts     = tf.placeholder(tf.int32, (None,))
        self.rew      = tf.placeholder(tf.float32, (None,))
        self.l_rate   = tf.placeholder(tf.float32)
        self.training = tf.placeholder(tf.bool)
        

        flat = tf.layers.flatten(self.obs, name='flat')
        
        h1 = tf.layers.dense(flat, 128, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='h1')
        drop1 = tf.layers.dropout(h1, training=self.training, name='drop1')
        h2 = tf.layers.dense(drop1,   64, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='h2')
        drop2 = tf.layers.dropout(h2, training=self.training, name='drop2')
        h3 = tf.layers.dense(drop2,   32, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='h3')
        
        drop3 = tf.layers.dropout(h3, training=self.training, name='dropout')
        
        # This needs to have linear activations to conform to the "unscaled log probabilities" requirement of `sparse_softmax_cross_entropy_loss_with_logits()`.
        out = tf.layers.dense(drop3, self.n_actions, kernel_initializer=tf.random_normal_initializer(), name='out')
        
        # Compute normalized probabilities associated with each action.
        self.probabilities = tf.nn.softmax(out)
        
        # Returns softmax cross entropy loss.
        self.neg_log_prob = tf.losses.sparse_softmax_cross_entropy(logits=out, labels=self.acts)
        self.alt_neg_log_prob = tf.reduce_sum(-tf.log(self.probabilities) * tf.one_hot(self.acts, self.n_actions), axis=1)
        self.loss = self.neg_log_prob * self.rew
        
        # Set optimizer.
        self.train_op = tf.train.AdamOptimizer(self.l_rate).minimize(self.loss)
    
    def save(self, path):
        self.saver.save(self.sess, path)
        
    def load(self, path):
        self.saver.restore(self.sess, path)
    
    def choose_action(self, obs):
        # Compute probabilities associated with each action.
        prob_weights = self.sess.run(self.probabilities, feed_dict={
            self.obs:      np.array(obs).reshape(-1, 8),
            self.training: False
        })
        
        # Choose action based on computed probabilities.
        return np.random.choice(range(prob_weights.shape[1]), p=prob_weights.ravel())
    
    def train(self, act, obs, rew, l_rate):
        _, neg, alt_neg, loss = self.sess.run([self.train_op,self.neg_log_prob,self.alt_neg_log_prob,self.loss], feed_dict={
            self.obs:      np.array(obs).reshape(-1, 8),
            self.acts:     act,
            self.rew:      rew,
            self.l_rate:   l_rate,
            self.training: True
        })
        return loss,neg,alt_neg


In [5]:
class AgentHandler(object):
    def __init__(self, agent, env, path='./.model.ckpt'):
        self.agent = agent
        self.env = env
        
        self.saver = tf.train.Saver() # saves all variables
        self.path = path
        
    def run(self, train_func, rollout=100, l_rate=0.001, render=False, **kwargs):
        assert isinstance(train_func, str) and train_func.startswith('train_'), \
               'invalid train_func name specified'
        getattr(self, train_func)(self.rollout(rollout,render), l_rate, **kwargs)
        
        # Close the display window
        if render: self.env.close()
        
    def train_rsample(self, batch, l_rate, num_epochs=50, mini_batch_size=100):
        for _ in range(num_epochs):
            indices = np.random.randint(len(batch['rew']), size=mini_batch_size)
            self.agent.train([batch['act'][i] for i in indices],
                             [batch['obs'][i] for i in indices],
                             [batch['rew'][i] for i in indices],
                             l_rate)
        
    def train_all(self, batch, l_rate):
        loss,neg,alt_neg = self.agent.train(batch['act'],
                         batch['obs'],
                         batch['rew'],
                         l_rate)
#         print('loss:{}     softmax:{}    alt_version:{}'.format(loss,neg,alt_neg))        
#         print('loss:{}'.format(np.mean(loss)))
    
    def process_rewards(self, rewards, decay=0.99):
        discounted = np.zeros_like(rewards)
        running_reward = 0
        
        for idx in reversed(range(len(rewards))):
            running_reward += rewards[idx]
            running_reward *= decay
            discounted[idx] = running_reward
            
        discounted -= np.mean(discounted)
        if np.std(discounted) != 0:
            discounted /= np.std(discounted)
        return discounted.tolist()

    def save(self):
        self.agent.save(self.path)
        
    def load(self):
        self.agent.load(self.path)
            
    def rollout(self, count, render):
        batch = {'act': [], 'obs': [], 'rew': []}
        
        for episode in range(count):
            # Stores all the stuff
            history = {'act': [], 'obs': [], 'rew': []}
            
            # Get observation of initial state of environment.
            obs_prev = self.env.reset()
            
            # Randomly choose first action in order to create first difference frame.
            obs_curr, reward, done, _ = self.env.step(env.action_space.sample())
            
     
            while not done:
                
                # Make difference frame.
                diff_frame, obs_prev = obs_curr - obs_prev, obs_curr
                
        
                if render: self.env.render()
        
                # Agent chooses action based on difference frame.
                action = self.agent.choose_action(diff_frame)
        
                # Take action in environment.
                obs_curr, reward, done, _ = self.env.step(action)
                
                history['act'].append(action)
                history['obs'].append(diff_frame)
                history['rew'].append(reward)
                
            # Process rewards per episode.
            total_reward = sum(history['rew'])
            history['rew'] = self.process_rewards(history['rew'])
            
            
            # Add episode to batch.
            for key in batch:
                batch[key].extend(history[key])
                
        return batch

In [22]:
tf.reset_default_graph()
env = gym.make('LunarLander-v2')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [23]:
agent = PolicyAgent()

In [24]:
handler = AgentHandler(agent, env, '.models/lunar.cpt')

In [27]:
while(True):
    for _ in range(100):
        handler.run('train_all', rollout=30, l_rate=0.01, render=False)
    print('Completed 100 Training Iterations\n')
    handler.save()

Completed 100 Training Iterations

Completed 100 Training Iterations

Completed 100 Training Iterations

Completed 100 Training Iterations

Completed 100 Training Iterations



KeyboardInterrupt: 

In [25]:
handler.load()

INFO:tensorflow:Restoring parameters from .models/lunar.cpt


In [21]:
handler.save()